<a href="https://colab.research.google.com/github/rifkybujana/KOPSI/blob/Research%2FModel/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INSTALLATION

In [ ]:
!pip install tokenizers

     |████████████████████████████████| 3.3MB 5.2MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 5.2MB/s 
     |████████████████████████████████| 901kB 29.4MB/s 


In [ ]:
!pip install sagemaker

     |████████████████████████████████| 419kB 5.2MB/s 
     |████████████████████████████████| 133kB 51.3MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 7.7MB 41.2MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 143kB 33.2MB/s 
  Created wheel for sagemaker: filename=sagemaker-2.47.2.post0-py2.py3-none-any.whl size=584489 sha256=328746a4f9cae8a609407f604770e1f24933b82edbd36a3d4311a42bc4c01b89
  Stored in directory: /root/.cache/pip/wheels/84/f3/54/92432abfa686a1d1b0a5f7f1a9310112307b7576118f12ebe5
  Created wheel for protobuf3-to-dict: filename=protobuf3_to_dict-0.1.5-cp37-none-any.whl size=4031 sha256=a84a0c437e55a7485be14ba4e17b9c0a34b29283f4a3a8e2abaf8f60b37c65a3
  Stored in directory: /root/.cache/pip/wheels/37/42/d8/1609d310cabebc2cf60eca038fa2b0c8503412963734a6fc31
Successfully built sagemaker protobuf3-to-dict
ERROR: reques

In [ ]:
!pip install modules

  Created wheel for modules: filename=modules-1.0.0-cp37-none-any.whl size=1220 sha256=8c1792184944d3b03e7527b9c7b36711ac4376e58c82b72f3876a8f7ba2b2c30
  Stored in directory: /root/.cache/pip/wheels/5d/11/f8/026012647260b08e0ff34c936511eb3c219dceb0ccea24ddb0
Successfully built modules


## CODE

### Analyzing FacQA dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("train_preprocess.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2495 entries, 0 to 2494
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   question   2495 non-null   object
 1   passage    2495 non-null   object
 2   seq_label  2495 non-null   object
dtypes: object(3)
memory usage: 58.6+ KB


In [ ]:
df.describe()

,question,passage,seq_label
count,2495,2495,2495
unique,2412,1216,2077
top,"['Siapa', 'nama', 'presiden', 'Indonesia', 'se...","['Pemilu', 'pertama', 'multikandidat', 'ini', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
freq,7,17,6


In [ ]:
df.head()

,question,passage,seq_label
0,"['Kelompok', 'apakah', 'yang', 'menyatakan', '...","['Lewat', 'telepon', 'ke', 'kantor', 'berita',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"['Di', 'pulau', 'manakah', ',', 'PT', 'Mutiara...","['Sejak', 'tahun', '1980', '-', 'an', 'di', 'K...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['Apakah', 'warna', 'pasir', 'pada', 'pesisir'...","['Titik', 'kedua', 'adalah', 'Pantai', 'Baturu...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Apa', 'nama', 'perusahaan', 'China', 'yang',...","['Sichuan', ',', 'Kompas', '-', 'Pemerintah', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['Siapa', 'yang', 'akan', 'memindahkan', 'Patu...","['Jakarta', ',', 'Kompas', '-', 'Patung', 'Kar...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


Jika diperhatikan, dataset yang kita miliki semuanya berbentuk teks, sedangkan seharusnya masing masing kata disimpan dalam sebuah array. Oleh karena itu kita perlu mengubahnya menjadi array terlebih dahulu.

In [ ]:
from ast import literal_eval

In [ ]:
df['question'] = df['question'].apply(literal_eval)
df['passage'] = df['passage'].apply(literal_eval)
df['seq_label'] = df['seq_label'].apply(literal_eval)

In [ ]:
df.head()

,question,passage,seq_label
0,"[Kelompok, apakah, yang, menyatakan, bertanggu...","[Lewat, telepon, ke, kantor, berita, lokal, Cu...","[O, O, O, O, O, O, O, O, O, O, B, I, O, O, O, ..."
1,"[Di, pulau, manakah, ,, PT, Mutiara, Indopearl...","[Sejak, tahun, 1980, -, an, di, Kota, Bau-Bau,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[Apakah, warna, pasir, pada, pesisir, pantai, ...","[Titik, kedua, adalah, Pantai, Baturumah, sepa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[Apa, nama, perusahaan, China, yang, bekerja, ...","[Sichuan, ,, Kompas, -, Pemerintah, Indonesia,...","[O, O, O, O, O, O, O, O, O, B, I, I, I, O, O, ..."
4,"[Siapa, yang, akan, memindahkan, Patung, Karti...","[Jakarta, ,, Kompas, -, Patung, Kartini, di, J...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
' '.join(df['passage'][0])

'Lewat telepon ke kantor berita lokal Current News Service , Hezb-ul Mujahedeen , kelompok militan Kashmir yang terbesar , menyatakan bertanggung jawab atas ledakan di Srinagar .'

In [ ]:
def seq_to_index(seq_label):
    indexes = []
    for i in range(len(seq_label)):
        if seq_label[i] != 'O':
            indexes.append(i)

    indexes = [indexes[0], indexes[-1]]

    return indexes

In [ ]:
df['seq_label'] = df['seq_label'].apply(seq_to_index)

In [ ]:
df.head()

,question,passage,seq_label
0,"[Kelompok, apakah, yang, menyatakan, bertanggu...","[Lewat, telepon, ke, kantor, berita, lokal, Cu...","[10, 11]"
1,"[Di, pulau, manakah, ,, PT, Mutiara, Indopearl...","[Sejak, tahun, 1980, -, an, di, Kota, Bau-Bau,...","[30, 31]"
2,"[Apakah, warna, pasir, pada, pesisir, pantai, ...","[Titik, kedua, adalah, Pantai, Baturumah, sepa...","[19, 20]"
3,"[Apa, nama, perusahaan, China, yang, bekerja, ...","[Sichuan, ,, Kompas, -, Pemerintah, Indonesia,...","[9, 12]"
4,"[Siapa, yang, akan, memindahkan, Patung, Karti...","[Jakarta, ,, Kompas, -, Patung, Kartini, di, J...","[18, 20]"


Lakukan hal yang sama pada validation dataset

In [ ]:
val = pd.read_csv("val_preprocess.csv")

In [ ]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   question   311 non-null    object
 1   passage    311 non-null    object
 2   seq_label  311 non-null    object
dtypes: object(3)
memory usage: 7.4+ KB


In [ ]:
val.describe()

,question,passage,seq_label
count,311,311,311
unique,309,273,295
top,"['Siapa', 'Wakil', 'Presiden', 'Indonesia', '?']","['Stockholm', ',', 'Rabu', '-', 'Warga', 'Pera...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
freq,3,3,2


In [ ]:
val.head()

,question,passage,seq_label
0,"['Apa', 'nama', 'latin', 'dari', 'harimau', 'S...","['Selain', 'orangutan', ',', 'satwa', 'yang', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"['Berapa', 'penumpang', 'yang', 'turut', 'dala...","['Pesawat', 'dengan', 'nomor', 'penerbangan', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['Kepala', 'Badan', 'apakah', 'Mohamed', 'ElBa...","['Pernyataan', 'tersebut', 'disampaikan', 'Lar...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Berapa', 'kalikah', 'Lin', 'Dan', 'gagal', '...","['Tak', 'seperti', 'yang', 'diharapkan', ',', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['Apakah', 'nama', 'fakultas', 'tempat', 'Drh'...","['Yogyakarta', ',', 'Kompas', '-', 'Drh', 'Yud...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [ ]:
val['question'] = val['question'].apply(literal_eval)
val['passage'] = val['passage'].apply(literal_eval)
val['seq_label'] = val['seq_label'].apply(literal_eval)

In [ ]:
val.head()

,question,passage,seq_label
0,"[Apa, nama, latin, dari, harimau, Sumatera, ?]","[Selain, orangutan, ,, satwa, yang, berada, TN...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B, ..."
1,"[Berapa, penumpang, yang, turut, dalam, penerb...","[Pesawat, dengan, nomor, penerbangan, GA, -, 1...","[O, O, O, O, O, O, O, O, O, B, I, O, O, O, O, ..."
2,"[Kepala, Badan, apakah, Mohamed, ElBaradei, ?]","[Pernyataan, tersebut, disampaikan, Larijani, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B, ..."
3,"[Berapa, kalikah, Lin, Dan, gagal, memainkan, ...","[Tak, seperti, yang, diharapkan, ,, partai, im...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Apakah, nama, fakultas, tempat, Drh, Yuda, He...","[Yogyakarta, ,, Kompas, -, Drh, Yuda, Heru, Fi...","[O, O, O, O, O, O, O, O, O, O, O, B, I, I, O, ..."


In [ ]:
' '.join(val['question'][0])

'Apa nama latin dari harimau Sumatera ?'

In [ ]:
val['seq_label'] = val['seq_label'].apply(seq_to_index)

In [242]:
val.head()

,question,passage,seq_label
0,"[Apa, nama, latin, dari, harimau, Sumatera, ?]","[Selain, orangutan, ,, satwa, yang, berada, TN...","[14, 16]"
1,"[Berapa, penumpang, yang, turut, dalam, penerb...","[Pesawat, dengan, nomor, penerbangan, GA, -, 1...","[9, 10]"
2,"[Kepala, Badan, apakah, Mohamed, ElBaradei, ?]","[Pernyataan, tersebut, disampaikan, Larijani, ...","[14, 20]"
3,"[Berapa, kalikah, Lin, Dan, gagal, memainkan, ...","[Tak, seperti, yang, diharapkan, ,, partai, im...","[33, 34]"
4,"[Apakah, nama, fakultas, tempat, Drh, Yuda, He...","[Yogyakarta, ,, Kompas, -, Drh, Yuda, Heru, Fi...","[11, 13]"


### Train

In [248]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [249]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(df['passage'].values.tolist(), df['question'].values.tolist(), truncation=True, padding=True, is_split_into_words=True)
val_encodings = tokenizer(val['passage'].values.tolist(), val['question'].values.tolist(), truncation=True, padding=True, is_split_into_words=True)

In [250]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i][0]))
        end_positions.append(encodings.char_to_token(i, answers[i][1]))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, df['seq_label'])
add_token_positions(val_encodings, val['seq_label'])

In [251]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [252]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

In [253]:
results, text = model.eval_model(val_encodings)

AttributeError: ignored